## Построение baseline-решений

В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:

   1. Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем  большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.

2. Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.

3. Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.

4. Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.

Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество. На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель. Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами). Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения и оценки baseline-моделей!

Можно (но не обязательно) рассмотреть следующий набор алгоритмов:

- Линейная модель (например, реализация sklearn.linear_model.RidgeClassifier)
- Случайный лес (например, реализация sklearn.ensemble.RandomForestClassifier)
- Градиентный бустинг (например, реализация sklearn.ensemble.GradientBoostingClassifier)

In [215]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, model_selection, metrics, linear_model, ensemble, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [250]:
train = pd.read_csv('Train.csv')
train

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,label
0,NaN,NaN,NaN,NaN,NaN,805.0,7.0,NaN,NaN,NaN,...,fXLavGi,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,854.0,7.0,NaN,NaN,NaN,...,UulNbBP,LM8l689qOp,NaN,NaN,Qu4f,02N6s8f,R4y5gQQWY8OodqDV,am7c,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,560.0,21.0,NaN,NaN,NaN,...,s4wQZmh,LM8l689qOp,NaN,ELof,Qcbd,RAYp,F2FyR07IdsN7I,am7c,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,14.0,0.0,NaN,NaN,NaN,...,6iNOEra,LM8l689qOp,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
4,NaN,NaN,NaN,NaN,NaN,1806.0,7.0,NaN,NaN,NaN,...,uyDq6g3,LM8l689qOp,NaN,ELof,TNEC,02N6s8f,ib5G6X1eUxUn6,am7c,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,NaN,NaN,NaN,NaN,NaN,721.0,7.0,NaN,NaN,NaN,...,i9SE4Qn,LM8l689qOp,NaN,xG3x,Aoh3,RAYp,F2FyR07IdsN7I,am7c,NaN,-1
29996,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,LiUPLfR,LM8l689qOp,NaN,NaN,szEZ,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
29997,NaN,NaN,NaN,NaN,NaN,1897.0,7.0,NaN,NaN,NaN,...,k0gYyp5,LM8l689qOp,NaN,kG3k,Qu4f,RAYp,F2FyR07IdsN7I,mj86,NaN,1
29998,NaN,NaN,NaN,NaN,NaN,700.0,0.0,NaN,NaN,NaN,...,8C02pc5,LM8l689qOp,NaN,NaN,Xa3G,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


In [237]:
train.iloc[:,230:231]

,label
0,-1
1,-1
2,-1
3,-1
4,1
...,...
29995,-1
29996,-1
29997,1
29998,-1


In [264]:
# для кодирования категориальных признаков используется LabelEncoder
def label_encoder(data):
    
    labelencoder = LabelEncoder()
    for i in range(190,230):
        data.iloc[:,i] = labelencoder.fit_transform(data.iloc[:,i].astype(str))
    return data

In [176]:
def undersampling(data):
    data_churn = data[data['label'] == 1]
    data_nochurn = data[data['label'] == -1]
    countFirst = len(data_churn)
    data_churn = data_churn.append(data_nochurn.iloc[:countFirst,:])
    data_churn = shuffle(data_churn)
    return data_churn.iloc[:,:230], data_churn.iloc[:,230:]

In [265]:
# заменяем пропуски на 0
train = train.fillna(0)
# уравновешиваем классы 1 и -1 
x, y = undersampling(train)
# кодируем категориальные признаки
x = label_encoder(x)

In [240]:
skf = model_selection.StratifiedKFold(n_splits=5)

In [273]:
# рассчет метрик по кросс-валидации
def get_model(clf, skf, x, y):
    roc_auc = []
    f1_score = []
    precision = []
    recall = []
    for train, test in skf.split(x,y):
        clf.fit(x.iloc[train,:], y[train])
        y_pred = clf.predict(x.iloc[test,:])
        y_pred_prob = clf.predict_proba(x.iloc[test,:])[:,1]
        precision.append(metrics.precision_score(y[test], y_pred))
        recall.append(metrics.recall_score(y[test], y_pred))
        f1_score.append(metrics.f1_score(y[test], y_pred))
        roc_auc.append(metrics.roc_auc_score(y[test], y_pred_prob))
        
    print('f1_score:', np.mean(f1_score))
    print('precision:', np.mean(precision))
    print('recall:', np.mean(recall))
    print('roc_auc:', np.mean(roc_auc))
        
    return f1_score, precision, recall

Рассмотрим несколько различных алгоритмов построения baseline решения. Основная метрика f1-score, вспомогательные precision и recall.

In [274]:
clf = ensemble.GradientBoostingClassifier(random_state = 12)
f1_score, precision, recall = get_model(clf, skf, x, y.label.values)

f1_score: 0.6724565573851634
precision: 0.6519447112973656
recall: 0.6946462309620205
roc_auc: 0.7254034303849225


In [266]:
clf = ensemble.RandomForestClassifier()
f1_score, precision, recall = get_model(clf, skf, x, y.label.values)

f1_score: 0.6378619564672879
precision: 0.6437310609190201
recall: 0.6322508193560825


In [233]:
clf = linear_model.RidgeClassifier()
f1_score, precision, recall = get_model(clf, skf, x, y.label.values)

C:\Users\chetv\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.85229e-18): result may not be accurate.
  overwrite_a=True).T
C:\Users\chetv\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.64036e-18): result may not be accurate.
  overwrite_a=True).T
C:\Users\chetv\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.63417e-18): result may not be accurate.
  overwrite_a=True).T
C:\Users\chetv\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=3.47196e-18): result may not be accurate.
  overwrite_a=True).T
C:\Users\chetv\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.62777e-18): result may not be accurate.
  overwrite_a=True).T


f1_score: 0.6239758260043338
precision: 0.6003709698968692
recall: 0.6498187777135146


In [235]:
clf = linear_model.LogisticRegression(solver='liblinear', max_iter=1000)
f1_score, precision, recall = get_model(clf, skf, x, y.label.values)

f1_score: 0.5981510499153765
precision: 0.5761166189632626
recall: 0.6221544245228456


In [234]:
clf = svm.SVC()
f1_score, precision, recall = get_model(clf, skf, x, y.label.values)

f1_score: 0.5207991957620154
precision: 0.528746945767973
recall: 0.5136128783497205


Вывод: наилучший результат из рассмотренных алгоритмов посторения baseline показали GradienBoosting и RandomForest. В дальнейшем решении именно у этих классификаторов будет производится настройка параметров. 